## TODO:

- Function to retrieve all Tasks
- Function to retrieve all Ingredients
- Function to retrieve all Container
- Function to retrieve all Tools
- Add Errors for Tool and Container Combination
- Select Field for all Tasks
- Select Field for all Ingredients, divided by Ingredients Type
- Select Field for Container
- Select Field for Tools
- Function that Checks Functionality of Container +  Tool
- Function that executes Motion with choosen Task, Ingredients, Container, Tools
- Function that creates a Task Tree with choosen Task, Ingredients, Container Tools
- Add Query Builder in Notebook?

In [2]:
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.designators.action_designator import *
from pycram.enums import Arms
from pycram.designators.object_designator import *
from pycram.designators.object_designator import BelieveObject
import pycram.helper as helper
from pycram.resolver.action.mixing import MixingActionSWRL
import pandas as pd
from ipywidgets import widgets

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [12]:
import ipywidgets as widgets
from IPython.display import display

# Define your function
def do_something(task, ingredients):
    print(f"Task: {task}, Ingredients: {', '.join(ingredients)}")

# Create dropdown widget for tasks
tasks = ['Task 1', 'Task 2', 'Task 3']
task_dropdown = widgets.Dropdown(
    options=tasks,
    description='Task:',
    disabled=False,
)

# Create select multiple widget for ingredients
ingredients = ['Ingredient 1', 'Ingredient 2', 'Ingredient 3']
ingredients_select = widgets.SelectMultiple(
    options=ingredients,
    description='Ingredients',
    disabled=False
)

# Create a button to execute the function
execute_button = widgets.Button(
    description='Execute',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to execute',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Define the function to be called when button is clicked
def on_button_clicked(b):
    print(1)
    selected_task = task_dropdown.value
    selected_ingredients = list(ingredients_select.value)
    if selected_task and selected_ingredients:
        do_something(selected_task, selected_ingredients)
    else:
        print("Please select both a task and at least one ingredient.")

# Bind the button click event to the function
execute_button.on_click(on_button_clicked)

# Display the widgets
display(task_dropdown, ingredients_select, execute_button)


Dropdown(description='Task:', options=('Task 1', 'Task 2', 'Task 3'), value='Task 1')

SelectMultiple(description='Ingredients', options=('Ingredient 1', 'Ingredient 2', 'Ingredient 3'), value=())

Button(description='Execute', icon='check', style=ButtonStyle(), tooltip='Click to execute')

In [4]:
spawning_poses = {
    'whisk': Pose([0.9, 0.6, 0.8], [0, 0, 0, 1]),
    'woodenspoon': Pose([0.7, 0.6, 0.8], [0, 0, 0, 1]),
    'big-bowl': Pose([-0.85, 0.9, 0.91], [0, 0, -1, -1]),
    'pot': Pose([-0.85, 0.9, 0.86], [0, 0, -1, -1])
}

In [5]:
world = BulletWorld()
world.set_gravity([0, 0, -9.8])
# plane = Object("floor", "environment", "plane.urdf", world=world)
robot = Object("pr2", "robot", "../../resources/" + robot_description.name + ".urdf")
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()
kitchen = Object("kitchen", "environment", "kitchen.urdf")
robot.set_joint_state(robot_description.torso_joint, 0.24)
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

Scalar element defined multiple times: limit


In [6]:
whisk = Object("whisk", "whisk", "whisk.stl", spawning_poses["whisk"])
wooden_spoon = Object("woodenspoon", "woodenspoon", "woodenspoon.stl", spawning_poses["woodenspoon"])
#fork = Object("fork", "fork", "fork.stl", spawning_poses['fork'])
big_bowl = Object("big-bowl", "big-bowl", "big-bowl.stl", spawning_poses["big-bowl"])
#pan = Object("pan", "pan", "pan.stl", spawning_poses['big-bowl'])
#pot = Object("pot", "pot", "pot.stl", spawning_poses['pot'])
whisk_BO = BelieveObject(names=["whisk"])
big_bowl_BO = BelieveObject(names=["big-bowl"])
wooden_spoon_BO = BelieveObject(names=["woodenspoon"])
#pan_BO = BelieveObject(names=["pan"])
#pot_BO = BelieveObject(names=["pot"])

Scalar element defined multiple times: limit


In [7]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.33]).resolve().perform()
    arm = "left"

    pickup_pose_knife = CostmapLocation(target=wooden_spoon_BO.resolve(), reachable_for=robot_desig).resolve()

    NavigateAction(target_locations=[pickup_pose_knife.pose]).resolve().perform()

    PickUpAction(object_designator_description=wooden_spoon_BO,
                 arms=pickup_pose_knife.reachable_arms,
                 grasps=["top"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    original_quaternion = [0, 0, 0, 1]
    rotation_axis = [0, 0, 1]

    rotation_quaternion = helper.axis_angle_to_quaternion(rotation_axis, 180)
    resulting_quaternion = helper.multiply_quaternions(original_quaternion, rotation_quaternion)

    nav_pose = Pose([-0.3, 0.9, 0.0], resulting_quaternion)

    NavigateAction(target_locations=[nav_pose]).resolve().perform()
    LookAtAction(targets=[big_bowl_BO.resolve().pose]).resolve().perform()
    mixing_resolver = MixingActionSWRL(object_designator_description=big_bowl_BO,
                                       object_tool_designator_description=wooden_spoon_BO,
                                       ingredients=["butter"],
                                       task="folding task",
                                       arms=["left"],
                                       grasps=["top"]).parameters_from_owl().perform()

[INFO] [1722935605.568373]: Adjusted target pose based on special knowledge for grasp: top


* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2-api-2.19.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-api-2.19.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/l

['folding']
[{'folding_rotation_shift': 90, 'repetitive_folding_rotation_shift': 22.5, 'radius_bounds': [0.7, 0.0]}]
Objects dimensions: [0.25047288024425507, 0.2504728802442552, 0.11716735297441483]
Tool dimensions: [0.09302983708121038, 0.09505533000775945, 0.2866043447799642]


* Owlready2 * Pellet took 0.9388718605041504 seconds
* Owlready * Reparenting FoldingTask(?x), Ingredient(?ing1), hasIngredient(?x, ?ing1), Motion(?motion), performMotion(?x, ?motion) -> FoldingMotion(?motion): {swrl.Imp} => {owl.Thing}
* Owlready * Reparenting mixing.motion: {mixing.Motion} => {mixing.FoldingMotion}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
